<a href="https://colab.research.google.com/github/Sanarazaaa/Personal-Projects-On-going-/blob/main/face_detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript

def capture_image():
    js = Javascript('''
        async function captureImage() {
            const video = document.createElement('video');
            video.style.display = 'none';
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            await video.play();

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            stream.getTracks().forEach(track => track.stop());
            video.remove();

            const data = canvas.toDataURL('image/jpeg');
            google.colab.kernel.invokeFunction('notebook.run_callback', [data], {});
        }
        captureImage();
    ''')
    display(js)

# This function will be called with the captured image data
def save_image(data):
    import IPython.display as display
    import base64
    from PIL import Image
    from io import BytesIO

    # Decode base64 image data
    image_data = base64.b64decode(data.split(',')[1])
    image = Image.open(BytesIO(image_data))
    image.save('/content/captured_image.jpg')
    display.display(image)

# Register the callback
from google.colab import output
output.register_callback('notebook.run_callback', save_image)

# Capture image
capture_image()


In [ ]:
# working

import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Read the captured image
image = cv2.imread('/content/captured_image.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Multi-scale detection using Haar Cascade
faces = []
scales = [1.05, 1.1, 1.2]  # Multiple scale factors for better detection
for scale in scales:
    detected_faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=scale,
        minNeighbors=3,  # Lowered for more faces, can increase for fewer false positives
        minSize=(30, 30)
    )
    faces.extend(detected_faces)

# Filter out duplicate faces using Intersection over Union (IoU)
def iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate the area of intersection
    inter_x1 = max(x1, x2)
    inter_y1 = max(y1, y2)
    inter_x2 = min(x1 + w1, x2 + w2)
    inter_y2 = min(y1 + h1, y2 + h2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - inter_area

    return inter_area / union_area if union_area > 0 else 0

# Remove duplicates based on IoU threshold
threshold = 0.3  # You can adjust this threshold to your liking
unique_faces = []

for face in faces:
    if not any(iou(face, existing_face) > threshold for existing_face in unique_faces):
        unique_faces.append(face)

# Draw rectangles around detected faces using Haar Cascade
for (x, y, w, h) in unique_faces:
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

# Show the output with face detections
cv2_imshow(image)
print(f"Detected {len(unique_faces)} face(s).")
